In [114]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import urllib.request
import sys
import pyodbc
from psycopg2 import connect
from geopy.geocoders import Nominatim


In [75]:
# read all the needed files
xls = pd.ExcelFile('2005-19_UK_local_and_regional_CO2_emissions.xlsx')
df1 = pd.read_excel(xls, 'Full dataset',skiprows=[0])
df2 = pd.read_excel(xls, 'Pollution Inventory',skiprows=[0])


In [76]:
# turning df to parquet
df1.to_parquet('UK_local_and_regional_CO2_emissions.parquet')
# change the column name so it will let me save as parquet
df2.columns = ['Local Authority Distract Name', 'Operator', 'Site', 'Postcode','Reference','Substance Name','2005 year','2006 year','2007 year','2008 year','2009 year','2010 year','2011 year','2012 year','2013 year','2014 year','2015 year','2016 year','2017 year','2018 year','2019 year']
df2 = df2.astype("string")
df2.to_parquet('UK_local_and_regional_CO2_emissions_pollution_inventory.parquet')

In [90]:
df1 = pd.read_csv('list.csv')
df2 = pd.read_csv('df_detail.csv')

In [97]:
# combining the two csv file into a df
df3 = pd.concat([df1, df2], axis=1)

In [98]:
# drop all the NA values and unnamed columns
df3 = df3.drop(columns = 'Unnamed: 0')
df3 = df3.dropna(how='any')

In [99]:
# get only a lowerletter name with no space 
df3['companyNameLetter'] = df3['companyName'].str.replace(" ","").str.replace("'","").str.lower()

In [111]:
# to get the post code from address string 
df3['postCode'] = [val[-1] for val in df3['address'].str.split(',')]


,companyName,companyNumber,status,link,address,companyType,incorporated,industryCode,companyNameLetter,postCode
0,\ COMPANY LTD,5060411,Active,https://find-and-update.company-information.se...,"85 Great Portland Street, London, England, W1W...",Private limited Company,2-Mar-04,99999 - Dormant Company,\companyltd,W1W 7LT
1,* LTD,5062633,Active,https://find-and-update.company-information.se...,"85 Great Portland Street, London, England, W1W...",Private limited Company,3-Mar-04,99999 - Dormant Company,*ltd,W1W 7LT
2,? LTD,5063820,Dissolved,https://find-and-update.company-information.se...,"85 Great Portland Street, London, England, W1W...",Private limited Company,4-Mar-04,99999 - Dormant Company,?ltd,W1W 7LT
3,*+ LTD,5153997,Active,https://find-and-update.company-information.se...,"Enterprise Court, Downmill Road, Bracknell, RG...",Private limited Company,15-Jun-04,62090 - Other information technology service a...,*+ltd,RG12 1QS
4,.CO LTD,8013348,Active,https://find-and-update.company-information.se...,"Metro House 57 Peper Road, Hunslet, Leeds, Yor...",Private limited Company,30-Mar-12,99999 - Dormant Company,.coltd,LS10 2RU
...,...,...,...,...,...,...,...,...,...,...
696,A&JM BUILDING LIMITED,12114375,Active,https://find-and-update.company-information.se...,"123a Allerton Road, Mossley Hill, Liverpool, E...",Private limited Company,22-Jul-19,43910 - Roofing activities,a&jmbuildinglimited,L18 2DD
697,A&S BABUSHKA LIMITED,12115551,Active,https://find-and-update.company-information.se...,"6 Dinsdale Road, London, United Kingdom, SE3 7RL",Private limited Company,22-Jul-19,56103 - Take-away food shops and mobile food s...,a&sbabushkalimited,SE3 7RL
698,A&A FASHIONS LTD,12119626,Dissolved,https://find-and-update.company-information.se...,"Unit 4 2nd Floor Imperial Building, East Park ...",Private limited Company,24-Jul-19,47710 - Retail sale of clothing in specialised...,a&afashionsltd,LE5 4QD
699,A & M SPITZER LTD,12121346,Active,https://find-and-update.company-information.se...,"55 Fairholt Road, London, England, N16 5EW",Private limited Company,24-Jul-19,68100 - Buying and selling of own real estate,a&mspitzerltd,N16 5EW


In [116]:
geolocator = Nominatim(timeout=10, user_agent="PDS")

In [152]:
# using geo package to get geo info
df3['geocode'] = df3['postCode'].apply(geolocator.geocode)

In [161]:
df3  = df3.dropna(subset=['geocode'])

In [163]:
# getting latitude and longtitude by geo 
df3['latitude'] = [i.latitude for i in df3['geocode']]
df3['longitude'] = [g.longitude for g in df3['geocode']]

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_31172/187059395.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['latitude'] = [i.latitude for i in df3['geocode']]
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_31172/187059395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['longitude'] = [g.longitude for g in df3['geocode']]


In [174]:
df3['geocode'] = df3['geocode'].astype(str)
df3.to_parquet("details.parquet")

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_31172/3785533160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['geocode'] = df3['geocode'].astype(str)


In [175]:
# data cleaning process
df4 = df1.drop(columns = 'Unnamed: 0')
df4['companyNameLetter'] = df4['companyName'].str.replace(" ","").str.replace("'","").str.lower()

,companyName,companyNumber,status,link
0,\ COMPANY LTD,5060411,Active,https://find-and-update.company-information.se...
1,* LTD,5062633,Active,https://find-and-update.company-information.se...
2,? LTD,5063820,Dissolved,https://find-and-update.company-information.se...
3,*+ LTD,5153997,Active,https://find-and-update.company-information.se...
4,.CO LTD,8013348,Active,https://find-and-update.company-information.se...
...,...,...,...,...
3995,A.A.C. SERVICES LIMITED,2433281,Active,https://find-and-update.company-information.se...
3996,AAC (SERVICES) SOUTHERN LIMITED,8649344,Active,https://find-and-update.company-information.se...
3997,AAC SERVICE (SUSSEX) LIMITED,7957625,Active,https://find-and-update.company-information.se...
3998,AAC SHIPPING LIMITED,12496624,Dissolved,https://find-and-update.company-information.se...


In [178]:
df4.to_parquet("list.parquet")

In [179]:
df_new1= pd.read_parquet('UK_local_and_regional_CO2_emissions.parquet', engine='pyarrow') 
df_new2= pd.read_parquet('UK_local_and_regional_CO2_emissions_pollution_inventory.parquet', engine='pyarrow') 
df_new3= pd.read_parquet('list.parquet', engine='pyarrow') 
df_new4= pd.read_parquet('details.parquet', engine='pyarrow') 

In [180]:
df_new1

,Region/Country,Second Tier Authority,Local Authority,Code,Year,Industry Electricity,Industry Gas,Industry 'Other Fuels',Large Industrial Installations,Agriculture,...,Net Emissions: Grassland,Net Emissions: Wetlands,Net Emissions: Settlements,Net Emissions: Harvested Wood Products,LULUCF Net Emissions,Grand Total,"Population ('000s, mid-year estimate)",Per Capita Emissions (t),Area (km2),Emissions per km2 (kt)
0,North East,Darlington,Darlington,E06000005,2005,50.550343,101.278940,30.562372,0.043209,6.256185,...,-7.414084,0.000000,4.829007,0.000000,-0.901525,816.721455,100.287,8.143842,197.4758,4.135805
1,North East,Darlington,Darlington,E06000005,2006,54.152213,86.084018,29.496473,0.063666,6.081723,...,-7.576054,0.000000,4.665830,0.000000,-1.375206,799.804119,101.509,7.879145,197.4758,4.050137
2,North East,Darlington,Darlington,E06000005,2007,51.184033,84.152662,29.794050,0.072018,5.470460,...,-7.666460,0.000000,4.518375,0.000000,-1.996378,775.703345,102.632,7.558104,197.4758,3.928093
3,North East,Darlington,Darlington,E06000005,2008,50.584058,84.407453,36.551114,0.053536,5.679644,...,-7.795103,0.000000,4.381341,0.000000,-2.259225,766.428412,103.694,7.391251,197.4758,3.881126
4,North East,Darlington,Darlington,E06000005,2009,44.538614,72.672287,25.337118,1.168834,5.472433,...,-7.876808,0.000000,4.255476,0.000000,-2.298309,693.880495,104.355,6.649231,197.4758,3.513750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6340,National Total,National Total,National Total,None,2015,25770.859174,15841.098828,17849.168446,39068.316953,5678.792334,...,-2980.136707,1288.188206,5242.401015,-2800.323524,-1241.080483,397152.155527,65110.034,6.099707,248717.5814,1.596800
6341,National Total,National Total,National Total,None,2016,20189.023416,15632.027333,17807.958518,32850.425389,5921.738730,...,-2966.778884,655.978682,5520.557590,-2528.312395,-1582.652723,373745.487306,65648.054,5.693169,248717.5814,1.502690
6342,National Total,National Total,National Total,None,2017,18946.657827,13786.016749,18153.528298,33085.378883,5849.611540,...,-3281.379365,635.234451,5421.232364,-2297.522522,-1872.517811,361598.080083,66040.229,5.475421,248717.5814,1.453850
6343,National Total,National Total,National Total,None,2018,17639.557245,14557.462169,17732.489233,31783.799051,5943.436251,...,-3440.406039,1051.379715,5426.048342,-2195.488809,-1383.288330,357415.811108,66435.550,5.379888,248717.5814,1.437035


In [181]:
df_new2

,Local Authority Distract Name,Operator,Site,Postcode,Reference,Substance Name,2005 year,2006 year,2007 year,2008 year,...,2010 year,2011 year,2012 year,2013 year,2014 year,2015 year,2016 year,2017 year,2018 year,2019 year
0,Aberdeen City,Arjo Wiggins Ltd,Stoneywood Mill,AB21 9AB,0,Carbon dioxide,0.0,0.0,69.533,59.867,...,61.208,60.835,60.913,63.393,63.3081,64.77,71.584,73.595,64.058,57.722
1,Aberdeen City,Scottish Water Contracting,Nigg STW,AB12 3LT,0,Carbon dioxide,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,21.0,14.0,14.0,15.0,16.0,15.0,0.0
2,Aberdeen City,Sita North East Ltd,Hill Of Tramaud Landfill Site,0,0,Carbon dioxide,0.0,0.0,0.0,0.0,...,0.0,12.0,11.9,10.3,0.0,0.0,0.0,0.0,0.0,0.0
3,Aberdeen City,United Fish Products Ltd,Aberdeen,AB12 3AY,0,Carbon dioxide,0.0,0.0,11.401146,10.858234,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aberdeenshire,BP Exploration Co Ltd,Cruden Bay,AB42 0QH,0,Carbon dioxide,0.0,0.0,15.0,13.37434,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2577,York,British Sugar Plc,York,YO26 6XF,BW9239IF,Carbon dioxide - 'thermal',0.0,57.291,80.64,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2578,York,Nestle UK Ltd,York,YO91 1XY,BO9298IQ,Carbon dioxide,0.0,0.0,0.0,0.0,...,32.698,30.95,26.668,26.778,30.582,29.554,25.671,24.799,31.684,32.353
2579,York,Nestle UK Ltd,York,YO91 1XY,BO9298IQ,Carbon dioxide - 'thermal',0.0,0.0,0.0,43.835,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2580,York,Yorkshire Water Services Ltd,York Naburn STW,YO23 2XD,27/24/0124,Carbon dioxide,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [182]:
df_new3

,companyName,companyNumber,status,link,companyNameLetter
0,\ COMPANY LTD,5060411,Active,https://find-and-update.company-information.se...,\companyltd
1,* LTD,5062633,Active,https://find-and-update.company-information.se...,*ltd
2,? LTD,5063820,Dissolved,https://find-and-update.company-information.se...,?ltd
3,*+ LTD,5153997,Active,https://find-and-update.company-information.se...,*+ltd
4,.CO LTD,8013348,Active,https://find-and-update.company-information.se...,.coltd
...,...,...,...,...,...
3995,A.A.C. SERVICES LIMITED,2433281,Active,https://find-and-update.company-information.se...,a.a.c.serviceslimited
3996,AAC (SERVICES) SOUTHERN LIMITED,8649344,Active,https://find-and-update.company-information.se...,aac(services)southernlimited
3997,AAC SERVICE (SUSSEX) LIMITED,7957625,Active,https://find-and-update.company-information.se...,aacservice(sussex)limited
3998,AAC SHIPPING LIMITED,12496624,Dissolved,https://find-and-update.company-information.se...,aacshippinglimited


In [183]:
df_new4

,companyName,companyNumber,status,link,address,companyType,incorporated,industryCode,companyNameLetter,postCode,geocode,latitude,longitude
0,\ COMPANY LTD,5060411,Active,https://find-and-update.company-information.se...,"85 Great Portland Street, London, England, W1W...",Private limited Company,2-Mar-04,99999 - Dormant Company,\companyltd,W1W 7LT,"City of Westminster, Greater London, England, ...",51.518470,-0.142400
1,* LTD,5062633,Active,https://find-and-update.company-information.se...,"85 Great Portland Street, London, England, W1W...",Private limited Company,3-Mar-04,99999 - Dormant Company,*ltd,W1W 7LT,"City of Westminster, Greater London, England, ...",51.518470,-0.142400
2,? LTD,5063820,Dissolved,https://find-and-update.company-information.se...,"85 Great Portland Street, London, England, W1W...",Private limited Company,4-Mar-04,99999 - Dormant Company,?ltd,W1W 7LT,"City of Westminster, Greater London, England, ...",51.518470,-0.142400
3,*+ LTD,5153997,Active,https://find-and-update.company-information.se...,"Enterprise Court, Downmill Road, Bracknell, RG...",Private limited Company,15-Jun-04,62090 - Other information technology service a...,*+ltd,RG12 1QS,"Western Industrial Area, Bracknell, Easthampst...",51.414889,-0.766616
4,.CO LTD,8013348,Active,https://find-and-update.company-information.se...,"Metro House 57 Peper Road, Hunslet, Leeds, Yor...",Private limited Company,30-Mar-12,99999 - Dormant Company,.coltd,LS10 2RU,"Leeds, West Yorkshire, Yorkshire and the Humbe...",53.774973,-1.522427
...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,A&JM BUILDING LIMITED,12114375,Active,https://find-and-update.company-information.se...,"123a Allerton Road, Mossley Hill, Liverpool, E...",Private limited Company,22-Jul-19,43910 - Roofing activities,a&jmbuildinglimited,L18 2DD,"Liverpool, Birkenhead, Wirral, North West Engl...",53.384776,-2.906783
697,A&S BABUSHKA LIMITED,12115551,Active,https://find-and-update.company-information.se...,"6 Dinsdale Road, London, United Kingdom, SE3 7RL",Private limited Company,22-Jul-19,56103 - Take-away food shops and mobile food s...,a&sbabushkalimited,SE3 7RL,"Royal Borough of Greenwich, London, Greater Lo...",51.482890,0.009910
698,A&A FASHIONS LTD,12119626,Dissolved,https://find-and-update.company-information.se...,"Unit 4 2nd Floor Imperial Building, East Park ...",Private limited Company,24-Jul-19,47710 - Retail sale of clothing in specialised...,a&afashionsltd,LE5 4QD,"City of Leicester, East Midlands, England, LE5...",52.635990,-1.102740
699,A & M SPITZER LTD,12121346,Active,https://find-and-update.company-information.se...,"55 Fairholt Road, London, England, N16 5EW",Private limited Company,24-Jul-19,68100 - Buying and selling of own real estate,a&mspitzerltd,N16 5EW,"London Borough of Hackney, Greater London, Eng...",51.569390,-0.083680


In [187]:
# passing data to postgreSQL database
conn = connect(
    dbname = "Initialdatabase",
    user = "shihao",
    host = "de166.c0imhubqi7yg.us-east-1.rds.amazonaws.com",
    password = "qwerty123")
cur = conn.cursor()
count =0
for index, row in df_new3.iterrows():
    cur.execute("INSERT INTO public.companyList (companyNameLetter, companyName, companyNumber, status, link) VALUES (%s,%s,%s,%s,%s)",(row.companyNameLetter, row.companyName, row.companyNumber,row.status,row.link))
    conn.commit()
    count +=1
    print("success"+str(count))    
cur.close()
conn.close()

success1
success2
success3
success4
success5
success6
success7
success8
success9
success10
success11
success12
success13
success14
success15
success16
success17
success18
success19
success20
success21
success22
success23
success24
success25
success26
success27
success28
success29
success30
success31
success32
success33
success34
success35
success36
success37
success38
success39
success40
success41
success42
success43
success44
success45
success46
success47
success48
success49
success50
success51
success52
success53
success54
success55
success56
success57
success58
success59
success60
success61
success62
success63
success64
success65
success66
success67
success68
success69
success70
success71
success72
success73
success74
success75
success76
success77
success78
success79
success80
success81
success82
success83
success84
success85
success86
success87
success88
success89
success90
success91
success92
success93
success94
success95
success96
success97
success98
success99
success100
success1

success756
success757
success758
success759
success760
success761
success762
success763
success764
success765
success766
success767
success768
success769
success770
success771
success772
success773
success774
success775
success776
success777
success778
success779
success780
success781
success782
success783
success784
success785
success786
success787
success788
success789
success790
success791
success792
success793
success794
success795
success796
success797
success798
success799
success800
success801
success802
success803
success804
success805
success806
success807
success808
success809
success810
success811
success812
success813
success814
success815
success816
success817
success818
success819
success820
success821
success822
success823
success824
success825
success826
success827
success828
success829
success830
success831
success832
success833
success834
success835
success836
success837
success838
success839
success840
success841
success842
success843
success844
success845
success846

success1460
success1461
success1462
success1463
success1464
success1465
success1466
success1467
success1468
success1469
success1470
success1471
success1472
success1473
success1474
success1475
success1476
success1477
success1478
success1479
success1480
success1481
success1482
success1483
success1484
success1485
success1486
success1487
success1488
success1489
success1490
success1491
success1492
success1493
success1494
success1495
success1496
success1497
success1498
success1499
success1500
success1501
success1502
success1503
success1504
success1505
success1506
success1507
success1508
success1509
success1510
success1511
success1512
success1513
success1514
success1515
success1516
success1517
success1518
success1519
success1520
success1521
success1522
success1523
success1524
success1525
success1526
success1527
success1528
success1529
success1530
success1531
success1532
success1533
success1534
success1535
success1536
success1537
success1538
success1539
success1540
success1541
success1542
succ

success2143
success2144
success2145
success2146
success2147
success2148
success2149
success2150
success2151
success2152
success2153
success2154
success2155
success2156
success2157
success2158
success2159
success2160
success2161
success2162
success2163
success2164
success2165
success2166
success2167
success2168
success2169
success2170
success2171
success2172
success2173
success2174
success2175
success2176
success2177
success2178
success2179
success2180
success2181
success2182
success2183
success2184
success2185
success2186
success2187
success2188
success2189
success2190
success2191
success2192
success2193
success2194
success2195
success2196
success2197
success2198
success2199
success2200
success2201
success2202
success2203
success2204
success2205
success2206
success2207
success2208
success2209
success2210
success2211
success2212
success2213
success2214
success2215
success2216
success2217
success2218
success2219
success2220
success2221
success2222
success2223
success2224
success2225
succ

success2826
success2827
success2828
success2829
success2830
success2831
success2832
success2833
success2834
success2835
success2836
success2837
success2838
success2839
success2840
success2841
success2842
success2843
success2844
success2845
success2846
success2847
success2848
success2849
success2850
success2851
success2852
success2853
success2854
success2855
success2856
success2857
success2858
success2859
success2860
success2861
success2862
success2863
success2864
success2865
success2866
success2867
success2868
success2869
success2870
success2871
success2872
success2873
success2874
success2875
success2876
success2877
success2878
success2879
success2880
success2881
success2882
success2883
success2884
success2885
success2886
success2887
success2888
success2889
success2890
success2891
success2892
success2893
success2894
success2895
success2896
success2897
success2898
success2899
success2900
success2901
success2902
success2903
success2904
success2905
success2906
success2907
success2908
succ

success3509
success3510
success3511
success3512
success3513
success3514
success3515
success3516
success3517
success3518
success3519
success3520
success3521
success3522
success3523
success3524
success3525
success3526
success3527
success3528
success3529
success3530
success3531
success3532
success3533
success3534
success3535
success3536
success3537
success3538
success3539
success3540
success3541
success3542
success3543
success3544
success3545
success3546
success3547
success3548
success3549
success3550
success3551
success3552
success3553
success3554
success3555
success3556
success3557
success3558
success3559
success3560
success3561
success3562
success3563
success3564
success3565
success3566
success3567
success3568
success3569
success3570
success3571
success3572
success3573
success3574
success3575
success3576
success3577
success3578
success3579
success3580
success3581
success3582
success3583
success3584
success3585
success3586
success3587
success3588
success3589
success3590
success3591
succ

In [189]:
# passing data to postgreSQL database
conn = connect(
    dbname = "Initialdatabase",
    user = "shihao",
    host = "de166.c0imhubqi7yg.us-east-1.rds.amazonaws.com",
    password = "qwerty123")
cur = conn.cursor()
count =0
for index, row in df_new4.iterrows():
    cur.execute("INSERT INTO public.companyDetail (companyNameLetter, companyName, companyNumber, address, companyType, incorporated, industryCode, postCode, latitude, longitude) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(row.companyNameLetter, row.companyName, row.companyNumber,row.address,row.companyType,row.incorporated,row.industryCode,row.postCode,row.latitude,row.longitude))
    conn.commit()
    count +=1
    print("success"+str(count))    
cur.close()
conn.close()

success1
success2
success3
success4
success5
success6
success7
success8
success9
success10
success11
success12
success13
success14
success15
success16
success17
success18
success19
success20
success21
success22
success23
success24
success25
success26
success27
success28
success29
success30
success31
success32
success33
success34
success35
success36
success37
success38
success39
success40
success41
success42
success43
success44
success45
success46
success47
success48
success49
success50
success51
success52
success53
success54
success55
success56
success57
success58
success59
success60
success61
success62
success63
success64
success65
success66
success67
success68
success69
success70
success71
success72
success73
success74
success75
success76
success77
success78
success79
success80
success81
success82
success83
success84
success85
success86
success87
success88
success89
success90
success91
success92
success93
success94
success95
success96
success97
success98
success99
success100
success1

In [199]:
# passing data to postgreSQL database
conn = connect(
    dbname = "Initialdatabase",
    user = "shihao",
    host = "de166.c0imhubqi7yg.us-east-1.rds.amazonaws.com",
    password = "qwerty123")
cur = conn.cursor()
count =0
for index, row in df_new2.iterrows():
    cur.execute("INSERT INTO public.pollutionInventory (LocalAuthorityDistractName,Operator, Site, Postcode, Reference, SubstanceName, year2005, year2006, year2007, year2008, year2009, year2010, year2011,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14],row[15],row[16],row[17],row[18],row[19],row[20]))
    conn.commit()
    count +=1
    print("success"+str(count))    
cur.close()
conn.close()

success1
success2
success3
success4
success5
success6
success7
success8
success9
success10
success11
success12
success13
success14
success15
success16
success17
success18
success19
success20
success21
success22
success23
success24
success25
success26
success27
success28
success29
success30
success31
success32
success33
success34
success35
success36
success37
success38
success39
success40
success41
success42
success43
success44
success45
success46
success47
success48
success49
success50
success51
success52
success53
success54
success55
success56
success57
success58
success59
success60
success61
success62
success63
success64
success65
success66
success67
success68
success69
success70
success71
success72
success73
success74
success75
success76
success77
success78
success79
success80
success81
success82
success83
success84
success85
success86
success87
success88
success89
success90
success91
success92
success93
success94
success95
success96
success97
success98
success99
success100
success1

success756
success757
success758
success759
success760
success761
success762
success763
success764
success765
success766
success767
success768
success769
success770
success771
success772
success773
success774
success775
success776
success777
success778
success779
success780
success781
success782
success783
success784
success785
success786
success787
success788
success789
success790
success791
success792
success793
success794
success795
success796
success797
success798
success799
success800
success801
success802
success803
success804
success805
success806
success807
success808
success809
success810
success811
success812
success813
success814
success815
success816
success817
success818
success819
success820
success821
success822
success823
success824
success825
success826
success827
success828
success829
success830
success831
success832
success833
success834
success835
success836
success837
success838
success839
success840
success841
success842
success843
success844
success845
success846

success1460
success1461
success1462
success1463
success1464
success1465
success1466
success1467
success1468
success1469
success1470
success1471
success1472
success1473
success1474
success1475
success1476
success1477
success1478
success1479
success1480
success1481
success1482
success1483
success1484
success1485
success1486
success1487
success1488
success1489
success1490
success1491
success1492
success1493
success1494
success1495
success1496
success1497
success1498
success1499
success1500
success1501
success1502
success1503
success1504
success1505
success1506
success1507
success1508
success1509
success1510
success1511
success1512
success1513
success1514
success1515
success1516
success1517
success1518
success1519
success1520
success1521
success1522
success1523
success1524
success1525
success1526
success1527
success1528
success1529
success1530
success1531
success1532
success1533
success1534
success1535
success1536
success1537
success1538
success1539
success1540
success1541
success1542
succ

success2143
success2144
success2145
success2146
success2147
success2148
success2149
success2150
success2151
success2152
success2153
success2154
success2155
success2156
success2157
success2158
success2159
success2160
success2161
success2162
success2163
success2164
success2165
success2166
success2167
success2168
success2169
success2170
success2171
success2172
success2173
success2174
success2175
success2176
success2177
success2178
success2179
success2180
success2181
success2182
success2183
success2184
success2185
success2186
success2187
success2188
success2189
success2190
success2191
success2192
success2193
success2194
success2195
success2196
success2197
success2198
success2199
success2200
success2201
success2202
success2203
success2204
success2205
success2206
success2207
success2208
success2209
success2210
success2211
success2212
success2213
success2214
success2215
success2216
success2217
success2218
success2219
success2220
success2221
success2222
success2223
success2224
success2225
succ

In [200]:
# passing data to postgreSQL database
conn = connect(
    dbname = "Initialdatabase",
    user = "shihao",
    host = "de166.c0imhubqi7yg.us-east-1.rds.amazonaws.com",
    password = "qwerty123")
cur = conn.cursor()
count =0
for index, row in df_new1.iterrows():
    cur.execute("INSERT INTO public.CO2emissions (RegionCountry, SecondTierAuthority, LocalAuthority, Code, Year, IndustryElectricity, IndustryGas, IndustryOtherFuels, LargeIndustrialInstallations, Agriculture, IndustryTotal) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10]))
    conn.commit()
    count +=1
    print("success"+str(count))    
cur.close()
conn.close()

success1
success2
success3
success4
success5
success6
success7
success8
success9
success10
success11
success12
success13
success14
success15
success16
success17
success18
success19
success20
success21
success22
success23
success24
success25
success26
success27
success28
success29
success30
success31
success32
success33
success34
success35
success36
success37
success38
success39
success40
success41
success42
success43
success44
success45
success46
success47
success48
success49
success50
success51
success52
success53
success54
success55
success56
success57
success58
success59
success60
success61
success62
success63
success64
success65
success66
success67
success68
success69
success70
success71
success72
success73
success74
success75
success76
success77
success78
success79
success80
success81
success82
success83
success84
success85
success86
success87
success88
success89
success90
success91
success92
success93
success94
success95
success96
success97
success98
success99
success100
success1

success756
success757
success758
success759
success760
success761
success762
success763
success764
success765
success766
success767
success768
success769
success770
success771
success772
success773
success774
success775
success776
success777
success778
success779
success780
success781
success782
success783
success784
success785
success786
success787
success788
success789
success790
success791
success792
success793
success794
success795
success796
success797
success798
success799
success800
success801
success802
success803
success804
success805
success806
success807
success808
success809
success810
success811
success812
success813
success814
success815
success816
success817
success818
success819
success820
success821
success822
success823
success824
success825
success826
success827
success828
success829
success830
success831
success832
success833
success834
success835
success836
success837
success838
success839
success840
success841
success842
success843
success844
success845
success846

success1460
success1461
success1462
success1463
success1464
success1465
success1466
success1467
success1468
success1469
success1470
success1471
success1472
success1473
success1474
success1475
success1476
success1477
success1478
success1479
success1480
success1481
success1482
success1483
success1484
success1485
success1486
success1487
success1488
success1489
success1490
success1491
success1492
success1493
success1494
success1495
success1496
success1497
success1498
success1499
success1500
success1501
success1502
success1503
success1504
success1505
success1506
success1507
success1508
success1509
success1510
success1511
success1512
success1513
success1514
success1515
success1516
success1517
success1518
success1519
success1520
success1521
success1522
success1523
success1524
success1525
success1526
success1527
success1528
success1529
success1530
success1531
success1532
success1533
success1534
success1535
success1536
success1537
success1538
success1539
success1540
success1541
success1542
succ

success2143
success2144
success2145
success2146
success2147
success2148
success2149
success2150
success2151
success2152
success2153
success2154
success2155
success2156
success2157
success2158
success2159
success2160
success2161
success2162
success2163
success2164
success2165
success2166
success2167
success2168
success2169
success2170
success2171
success2172
success2173
success2174
success2175
success2176
success2177
success2178
success2179
success2180
success2181
success2182
success2183
success2184
success2185
success2186
success2187
success2188
success2189
success2190
success2191
success2192
success2193
success2194
success2195
success2196
success2197
success2198
success2199
success2200
success2201
success2202
success2203
success2204
success2205
success2206
success2207
success2208
success2209
success2210
success2211
success2212
success2213
success2214
success2215
success2216
success2217
success2218
success2219
success2220
success2221
success2222
success2223
success2224
success2225
succ

success2826
success2827
success2828
success2829
success2830
success2831
success2832
success2833
success2834
success2835
success2836
success2837
success2838
success2839
success2840
success2841
success2842
success2843
success2844
success2845
success2846
success2847
success2848
success2849
success2850
success2851
success2852
success2853
success2854
success2855
success2856
success2857
success2858
success2859
success2860
success2861
success2862
success2863
success2864
success2865
success2866
success2867
success2868
success2869
success2870
success2871
success2872
success2873
success2874
success2875
success2876
success2877
success2878
success2879
success2880
success2881
success2882
success2883
success2884
success2885
success2886
success2887
success2888
success2889
success2890
success2891
success2892
success2893
success2894
success2895
success2896
success2897
success2898
success2899
success2900
success2901
success2902
success2903
success2904
success2905
success2906
success2907
success2908
succ

success3509
success3510
success3511
success3512
success3513
success3514
success3515
success3516
success3517
success3518
success3519
success3520
success3521
success3522
success3523
success3524
success3525
success3526
success3527
success3528
success3529
success3530
success3531
success3532
success3533
success3534
success3535
success3536
success3537
success3538
success3539
success3540
success3541
success3542
success3543
success3544
success3545
success3546
success3547
success3548
success3549
success3550
success3551
success3552
success3553
success3554
success3555
success3556
success3557
success3558
success3559
success3560
success3561
success3562
success3563
success3564
success3565
success3566
success3567
success3568
success3569
success3570
success3571
success3572
success3573
success3574
success3575
success3576
success3577
success3578
success3579
success3580
success3581
success3582
success3583
success3584
success3585
success3586
success3587
success3588
success3589
success3590
success3591
succ

success4192
success4193
success4194
success4195
success4196
success4197
success4198
success4199
success4200
success4201
success4202
success4203
success4204
success4205
success4206
success4207
success4208
success4209
success4210
success4211
success4212
success4213
success4214
success4215
success4216
success4217
success4218
success4219
success4220
success4221
success4222
success4223
success4224
success4225
success4226
success4227
success4228
success4229
success4230
success4231
success4232
success4233
success4234
success4235
success4236
success4237
success4238
success4239
success4240
success4241
success4242
success4243
success4244
success4245
success4246
success4247
success4248
success4249
success4250
success4251
success4252
success4253
success4254
success4255
success4256
success4257
success4258
success4259
success4260
success4261
success4262
success4263
success4264
success4265
success4266
success4267
success4268
success4269
success4270
success4271
success4272
success4273
success4274
succ

success4875
success4876
success4877
success4878
success4879
success4880
success4881
success4882
success4883
success4884
success4885
success4886
success4887
success4888
success4889
success4890
success4891
success4892
success4893
success4894
success4895
success4896
success4897
success4898
success4899
success4900
success4901
success4902
success4903
success4904
success4905
success4906
success4907
success4908
success4909
success4910
success4911
success4912
success4913
success4914
success4915
success4916
success4917
success4918
success4919
success4920
success4921
success4922
success4923
success4924
success4925
success4926
success4927
success4928
success4929
success4930
success4931
success4932
success4933
success4934
success4935
success4936
success4937
success4938
success4939
success4940
success4941
success4942
success4943
success4944
success4945
success4946
success4947
success4948
success4949
success4950
success4951
success4952
success4953
success4954
success4955
success4956
success4957
succ

success5558
success5559
success5560
success5561
success5562
success5563
success5564
success5565
success5566
success5567
success5568
success5569
success5570
success5571
success5572
success5573
success5574
success5575
success5576
success5577
success5578
success5579
success5580
success5581
success5582
success5583
success5584
success5585
success5586
success5587
success5588
success5589
success5590
success5591
success5592
success5593
success5594
success5595
success5596
success5597
success5598
success5599
success5600
success5601
success5602
success5603
success5604
success5605
success5606
success5607
success5608
success5609
success5610
success5611
success5612
success5613
success5614
success5615
success5616
success5617
success5618
success5619
success5620
success5621
success5622
success5623
success5624
success5625
success5626
success5627
success5628
success5629
success5630
success5631
success5632
success5633
success5634
success5635
success5636
success5637
success5638
success5639
success5640
succ

success6241
success6242
success6243
success6244
success6245
success6246
success6247
success6248
success6249
success6250
success6251
success6252
success6253
success6254
success6255
success6256
success6257
success6258
success6259
success6260
success6261
success6262
success6263
success6264
success6265
success6266
success6267
success6268
success6269
success6270
success6271
success6272
success6273
success6274
success6275
success6276
success6277
success6278
success6279
success6280
success6281
success6282
success6283
success6284
success6285
success6286
success6287
success6288
success6289
success6290
success6291
success6292
success6293
success6294
success6295
success6296
success6297
success6298
success6299
success6300
success6301
success6302
success6303
success6304
success6305
success6306
success6307
success6308
success6309
success6310
success6311
success6312
success6313
success6314
success6315
success6316
success6317
success6318
success6319
success6320
success6321
success6322
success6323
succ

Arjo Wiggins Ltd
